In [2]:
import os
import sys
import json
import torch
from tqdm import tqdm
from typing import List
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from peft import PeftConfig, PeftModel

/home/dshome/anaconda3/envs/auto_gptq_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Загрузка и сохранение на диск базовой модели.

In [5]:
base_model_path = 'models/'
adapter_path = 'models/'
tockenizer_path = 'models/'

In [3]:
# MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
# template_path = '/home/dshome/Projects/rulm/self_instruct/internal_prompts/saiga_v2.json'
# config = PeftConfig.from_pretrained(MODEL_NAME)

# base_model_config = AutoConfig.from_pretrained(config.base_model_name_or_path)

# base_model = AutoModelForCausalLM.from_pretrained(
#     config.base_model_name_or_path,
#     torch_dtype=base_model_config.torch_dtype,
#     load_in_8bit=True,
#     device_map="auto",
# )
# base_model.save_pretrained(base_model_path)

In [4]:
# Загружаем базовую модель. 
# Загружаем Адаптер.
# Загружаем токенайзер.

## Тестирование работы адаптера.

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(tockenizer_path, use_fast=False) #, padding_side='left')
# generation_config = GenerationConfig.from_pretrained(model_name, do_sample=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# config = PeftConfig.from_pretrained(base_model_path)
base_model_config = AutoConfig.from_pretrained(base_model_path)
torch_dtype = base_model_config.torch_dtype

In [8]:
quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch_dtype,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    ),

In [9]:
generation_config = GenerationConfig.from_pretrained(base_model_path,
                                                     do_sample=True)

In [10]:
# config = PeftConfig.from_pretrained(adapter_path)
# base_model_config = AutoConfig.from_pretrained(base_model_path)

# torch_dtype = base_model_config.torch_dtype

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    load_in_8bit=True,
    # quantization_config=quantization_config, # tесли не хотим 4-bit.
    torch_dtype=torch_dtype,
    device_map="auto",
)

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


In [11]:
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    adapter_name="adapter1_name",
    torch_dtype=torch_dtype
)

In [12]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (adapter1_name): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (adapter1_name): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (adapter1_name): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(
                in_features=4096, out_fe

In [13]:
print(model.active_adapter)

adapter1_name


In [ ]:
# Выполнить предсказание на модели.

# Посмотреть скрипты для загрузки модели и всех данных со своего google-drive.
# Надо еще посомотереть как зделать chackpoint на свой google-drive. 
# Чтобы не терять данные модели.
# Это в случае, если модель не влезал на мою видеокарту.  

In [14]:
system_prompt = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

In [15]:
user_message =  "Как приготовить борщ?"

In [16]:
s = f"<s>system\n{system_prompt}</s>\n" + \
    f"<s>user\n{user_message}</s>\n" + \
    f"<s>bot\n"

In [17]:
data = tokenizer(s,return_tensors="pt")

In [18]:
data = {k: v.to(device) for k, v in data.items()}

In [19]:
# Настроить параметры генерации текста.
output_ids = model.generate(**data, do_sample=True, temperature=0.9, max_new_tokens=512, top_p=0.95,
                             top_k=5, repetition_penalty=1.03, no_repeat_ngram_size=2, generation_config=generation_config)

In [20]:
# Переписать.
outputs = []
for sample_output_ids, sample_input_ids in zip(output_ids, data["input_ids"]):
    sample_output_ids = sample_output_ids[len(sample_input_ids):]
    sample_output = tokenizer.decode(sample_output_ids, skip_special_tokens=True)
    sample_output = sample_output.replace("</s>", "").strip()
    outputs.append(sample_output)
outputs[0]

'Борщ - это классический русский суп. Для его приготовления вам понадобится:\n\n- 1,5 кг красного или красно-желтого картофеля\n  \n  - 500 гр. куриного мяса\n    - косточка, измельченная до пюре\n    \n      - лук, перец, соль, свежие травы (по вкусу)\n        - масло\n        \nДля примечания: при использовании карри или кориандра, необходимо добавить 2 столовые ложки кокосового масла.\n1. Сначала необязательно нарежьте мясо на куски и обжарьte на сковороде до золотистого цвета. Оставите на огне, чтобы мясные жир и соединился с маслом.  В другой миску разбейте карротины и натрите в соли, перцем и свежей зеленью. Добавьем в маслину. Если вы хотите придать супу кисло-соленый вставку, додайте небольшое количество соевого соуса. Затем перемешай, посыпь солнечными яйцами, и дополните соленой водкой. Положите смесь на среднюю плиту. Возьмите маленькую кастрюлю, положи в неё 3-4 стакана воды и закройте крышкой, придерживая её с помощью крючка или крутящейся ручкой для супа. Выполнять это ну

Борщ — национальный украинский суп, который можно приготовлять различными способами в зависимости от местных традиций или вкусовых предпочтений. Вот несколько популярных рецептов:\n\n1. Основной реcipe: нарежьте картофель, морковь, лук, чеснок и кольца помидоров, затем обжарь их на оливковом масле. Добавьte воду, уксус и бульон, и варите в течение 20-30 минут до готовности. В конце добавляйте сливочное масло, зеленый луковичий перец и зелени (укроп или петрушку), и подавай.\n2. Украинская борщу: используй для начинки крутокоренники, курицу, говядину, конину или баранины, которые нужно нашинковать, обжарить на сковороде и дополнительно обсушить. Затем, следует применять основной прием, но с использованием мяса вместо картин.  Это более сложный и насыщенный реципи.  \n3. Борщу из грибов: сначала обжимай глубокий миска грыбной массы, добиваясь ее настойчивой текстуры. Потом следуют основные ингредиенты, а гренки можно додавать в конце, чтобы не испортить текст. Этот вариант является более легким и менее пикантным.   \n4. Соргоборщ: в качестве основы используется сорг, которое можно купить в магазинах или подготавить самостоятельно, смешав кукурузное зерно с мукой. Оно дает борщину более сладкую, менестреличную текстовую структуру, что делает ей более мягкой и нежной.    \n5. Супер-легкая борш: для этого варенье используют карта